# Домашняя работа #6

**Выводы**:
1. Google Colab

    1.1. Есть возможность сохранять файлы на Google Drive и подгружать их оттуда.
    
    1.2. Google без предупреждения делает recycle для инстансов как по таймауту, так и по потреблению памяти и процессора.
    
    1.3. Google Colab Pro никак не помог решить проблемы из 1.2
    
    1.4. Делать работу в Google Colab очень сложно. Долелывал ее на локальной машине.
    
    
2. Библиотека stanza работала значительно медленнее чем spacy. Пришлось переписать извлечение фич и переключить его на spacy.

3. Я попробовал собрать n-grams на своем корпусе, но их оказалось очень мало и добавление n-grams ухудшило результат на логистической регрессии. Код сбора n-grams остался в репозитории, но результаты их применения я не включил.

4. Я поработал с векторизацией данных и логистической регрессией.

5. Я попробовал Conditional random fields (но не смог добиться приемлемого результата с ним)

6. Попробовал подбор параметров при помощи RandomizedSearchCV

In [1]:
import csv
import json
import os.path

# I plan to have quite a number of pre-processed data that would be stored in 'data' folder
DATA_PATH = '/Users/dmytromindra/Projects/prj-nlp-2020/students/DmytroMindra/06-language-as-sequence/'

RAW_DATA_FILENAME = DATA_PATH + 'data/masc_sentences.tsv'
RAW_SENTENCES_FILENAME = DATA_PATH + 'data/stripped_masc_sentences.json'
BASELINE_FEATURES_FILENAME = DATA_PATH + 'data/baseline_features_dataset.json'
BASELINE_LABELS_FILENAME = DATA_PATH + 'data/baseline_labels_dataset.json'
IMPROVED_FEATURES_FILENAME = DATA_PATH + 'data/improved_features_dataset.json'
IMPROVED_LABELS_FILENAME = DATA_PATH + 'data/improved_labels_dataset.json'
RUN_ON_TEST_RAW_FILENAME = DATA_PATH + 'data/run-on-test.json'
RUN_ON_TEST_TOKENS_FILENAME = DATA_PATH + 'data/run_on_test_tokens_dataset.json'
RUN_ON_TEST_LABELS_FILENAME = DATA_PATH + 'data/run_on_test_labels_dataset.json'

In [2]:
def file_exists(filename):
  if os.path.isfile(filename):
    return True
  else:
    return False

In [3]:
def read_raw_data():
    """
    Reading the original file
    Stripping it from the meta information and saving in json
    """
    data = []
    with open(RAW_DATA_FILENAME) as tsvfile:
        reader = csv.DictReader(tsvfile, dialect='excel-tab')
        for row in reader:
            data.append(list(row.items())[6][1])

    return data


data = read_raw_data()

with open(RAW_SENTENCES_FILENAME, 'w') as outfile:
    json.dump(data, outfile)


print ('Total sentences in dataset:', len(data))


Total sentences in dataset: 85887


In [4]:
def read_raw_data_from_json():
    """
    Reading the original file
    Stripping it from the meta information and saving in json
    """
    data = []
    with open(RAW_DATA_FILENAME) as tsvfile:
        reader = csv.DictReader(tsvfile, dialect='excel-tab')
        for row in reader:
            data.append(list(row.items())[6][1])

    return data

masc_dataset = read_raw_data_from_json()
print ('Total sentences loaded:',len(masc_dataset))


Total sentences loaded: 85887


In [5]:
!pip install spacy

import spacy
nlp = spacy.load("en", disable=["ner"])

Глобальная переменная **BASELINE** переключает набор фич, которые **extract_token_features** извлекает из текста. Это не элегантное решение и в продакшн коде так делать не стоит.

In [29]:
from random import random, randrange

# a function for run-on sentence simulation
def decision(probability):
    return random() < probability

# a distance to the sentence root
def get_root_distance(word_no, sentence):
    max_distance = 8
    word = sentence.words[word_no]
    distance = 0

    while distance <= max_distance and word.head != 0:
        word = sentence.words[word.head - 1]
        distance += 1

    return distance


BASELINE  = False 

def extract_token_features(doc, token_no):
    token = doc[token_no]
    text = token.text
    
    is_upper = is_upper_text(text)
        
    left_token = ''
    left_pos = ''
    right_token = ''
    right_pos = ''
    is_right_upper = False
    is_sentence_end = False
    if token_no > 0:
                left_token = doc[token_no - 1].text
                left_pos = doc[token_no - 1].pos_
    if token_no <= len(doc) - 2:
                right_token = doc[token_no + 1].text
                right_pos = doc[token_no + 1].pos_

                if len(doc[token_no + 1].text) > 0 and \
                        doc[token_no + 1].text[0].isupper():
                    is_right_upper = True
    if token_no == len(doc) - 2 and right_token =='.':
                is_sentence_end = True
    if token_no == len(doc) - 1 and text != '.':
                is_sentence_end = True
    
    global BASELINE
    if BASELINE:
        features = {
                        'text': token.text,
                        'pos': token.pos_,
                        'lemma': token.lemma_,
                        'deprel': token.dep_
                    }
    else:
        features = {'text': token.text,
                        'pos': token.pos_,
                        'lemma': token.lemma_,
                        'deprel': token.dep_,
                        'head': token.head.text,
                        'is_upper': is_upper,
                        'left_token': left_token,
                        'left_pos': left_pos,
                        'right_token': right_token,
                        'right_pos': right_pos,
                        'is_right_upper': is_right_upper,
                        'is_sentence_end' : is_sentence_end
                        }
    return features

 

def is_upper_text(text):
    if len(text) > 0 and text[0].isupper():
        is_upper = True
    else:
        is_upper = False
    return is_upper

def extract_features(sentences):
    sentence_count = len(sentences)
    current_shift = 0
    sentence_end_positions = []
    
    text = concatenate_sentences(current_shift,sentence_count,sentence_end_positions,sentences)
    
    doc = nlp(text)    
    labels, tokens = calculate_text_labels_and_features(doc,sentence_count,sentence_end_positions)

    return (tokens, labels)


def concatenate_sentences(current_shift, sentence_count, sentence_end_positions, sentences):
    text = ''
    for sentence_no in range(sentence_count):
        sentence = sentences[sentence_no]
        if sentence_no != sentence_count - 1:
                if sentence.endswith('.'):
                    sentence = sentence[:-1]
                sentence_end_positions.append(current_shift + len(sentence) - 1)
                current_shift += len(sentence)
        if sentence_no > 0:
                lower_case_decision = decision(0.5)
                if lower_case_decision:
                    sentence = sentence[:1].lower() + sentence[1:]
        if text == '':
            text = sentence
        else:
            text = text + ' ' + sentence
    return text

def calculate_text_labels_and_features(doc, sentence_count, sentence_end_positions):
    current_sentnse = 0
    labels = []
    tokens = []
    for token_no in range(len(doc)):
        token = doc[token_no]
        features = extract_token_features(doc,token_no)

        label = False
        
        end_char = token.idx + len(token.text)

        if sentence_count > 1 and \
                current_sentnse < sentence_count - 1 and \
                end_char > sentence_end_positions[current_sentnse]:
            current_sentnse += 1
            label = True

        tokens.append(features)
        labels.append(label)
    return labels, tokens

 

def prepare_dataset(data):
    print('Processing data ...')
    labels = []
    tokens = []
    counter = 0
    while len(data) > 0:

        batch_size = randrange(4) + 1
        batch = data[:batch_size]
        data = data[batch_size:]
        dataset = extract_features(batch)
        tokens.append(dataset[0])
        labels.append(dataset[1])
        counter += 1
        if counter % 100 == 0:
            print('to go', len(data))

    return tokens, labels

In [30]:

extract_features(["If you don’t need a particular component of the pipeline – for example, the tagger or the parser, you can disable loading it."])

([{'text': 'If',
   'pos': 'SCONJ',
   'lemma': 'if',
   'deprel': 'mark',
   'head': 'need',
   'is_upper': True,
   'left_token': '',
   'left_pos': '',
   'right_token': 'you',
   'right_pos': 'PRON',
   'is_right_upper': False,
   'is_sentence_end': False},
  {'text': 'you',
   'pos': 'PRON',
   'lemma': '-PRON-',
   'deprel': 'nsubj',
   'head': 'need',
   'is_upper': False,
   'left_token': 'If',
   'left_pos': 'SCONJ',
   'right_token': 'do',
   'right_pos': 'AUX',
   'is_right_upper': False,
   'is_sentence_end': False},
  {'text': 'do',
   'pos': 'AUX',
   'lemma': 'do',
   'deprel': 'aux',
   'head': 'need',
   'is_upper': False,
   'left_token': 'you',
   'left_pos': 'PRON',
   'right_token': 'n’t',
   'right_pos': 'PART',
   'is_right_upper': False,
   'is_sentence_end': False},
  {'text': 'n’t',
   'pos': 'PART',
   'lemma': 'not',
   'deprel': 'neg',
   'head': 'need',
   'is_upper': False,
   'left_token': 'do',
   'left_pos': 'AUX',
   'right_token': 'need',
   'right_p

In [17]:
BASELINE = True

baseline_features = []
baseline_labels = []

if not file_exists(BASELINE_FEATURES_FILENAME) and not file_exists(BASELINE_LABELS_FILENAME):
  baseline_features, baseline_labels = prepare_dataset(masc_dataset)

  with open(BASELINE_FEATURES_FILENAME, 'w') as outfile:
    json.dump(baseline_features, outfile)
    
  with open(BASELINE_LABELS_FILENAME, 'w') as outfile:
      json.dump(baseline_labels, outfile)

  print ('Dumped baseline_features and baseline_labels')
else:
  print ('Using existing files:')

  with open(BASELINE_FEATURES_FILENAME) as json_file:
      for sentence in json.load(json_file):
          for token in sentence:
              baseline_features.append(token)


  with open(BASELINE_LABELS_FILENAME) as json_file:
      for sentence in json.load(json_file):
          for label in sentence:
              baseline_labels.append(label)

print ('Total features loaded:', len(baseline_features))

Processing data ...
to go 85648
to go 85380
to go 85128
to go 84872
to go 84625
to go 84378
to go 84137
to go 83875
to go 83638
to go 83400
to go 83160
to go 82903
to go 82654
to go 82438
to go 82192
to go 81939
to go 81701
to go 81447
to go 81187
to go 80941
to go 80678
to go 80414
to go 80161
to go 79922
to go 79683
to go 79427
to go 79178
to go 78938
to go 78692
to go 78446
to go 78184
to go 77923
to go 77667
to go 77430
to go 77184
to go 76931
to go 76663
to go 76418
to go 76177
to go 75922
to go 75683
to go 75437
to go 75213
to go 74976
to go 74715
to go 74453
to go 74186
to go 73934
to go 73666
to go 73423
to go 73179
to go 72924
to go 72694
to go 72437
to go 72171
to go 71917
to go 71668
to go 71422
to go 71153
to go 70896
to go 70658
to go 70409
to go 70162
to go 69901
to go 69649
to go 69396
to go 69161
to go 68901
to go 68660
to go 68418
to go 68172
to go 67910
to go 67673
to go 67422
to go 67185
to go 66941
to go 66683
to go 66436
to go 66186
to go 65940
to go 65686
to go 65

In [24]:
baseline_labels[2]

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False]

# Baseline: LogisticRegression on a small set of features

In [25]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score

features_data = []
labels_data = []


for sentence in baseline_features:
    features_data.extend(sentence)

for sentence in baseline_labels:
    labels_data.extend(sentence)

total_size = len(features_data)
trainset_size = round(total_size * 0.7)
testset_size = total_size - trainset_size

train_features = features_data[:trainset_size]
test_features = features_data[trainset_size:]

print ('Trainset size', len(train_features))
print('Evalset size', len(test_features))

train_labels = labels_data[:trainset_size]
test_labels = labels_data[trainset_size:]

vectorizer = DictVectorizer()
vec = vectorizer.fit(train_features)
print("Total number of features: ", len(vec.get_feature_names()))

train_features_vectorized = vec.transform(train_features)
test_features_vectorized = vec.transform(test_features)

lrc = LogisticRegression(random_state=42, solver="sag", multi_class="multinomial",
                         max_iter=100, verbose=1, n_jobs = -1)
lrc.fit(train_features_vectorized, train_labels)

Trainset size 1973926
Evalset size 845968
Total number of features:  152092


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


convergence after 74 epochs took 142 seconds


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.4min finished


LogisticRegression(multi_class='multinomial', random_state=42, solver='sag',
                   verbose=1)

In [26]:
predicted_labels = lrc.predict(test_features_vectorized)
print(classification_report(test_labels, predicted_labels))

              precision    recall  f1-score   support

       False       0.98      1.00      0.99    830306
        True       0.56      0.01      0.02     15662

    accuracy                           0.98    845968
   macro avg       0.77      0.51      0.51    845968
weighted avg       0.97      0.98      0.97    845968



In [0]:
scores = cross_val_score(lrc, train_features_vectorized, train_labels, cv=5)
print ('Cross validation:', scores)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


convergence after 76 epochs took 91 seconds


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.5min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


convergence after 75 epochs took 86 seconds


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.4min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


...

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.4min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


convergence after 76 epochs took 85 seconds


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.4min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


convergence after 76 epochs took 85 seconds
Cross validation: [0.98208103 0.98203843 0.98209105 0.98203593 0.98207602]


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.4min finished


In [33]:
BASELINE = False

improved_features = []
improved_labels = []

if not file_exists(IMPROVED_FEATURES_FILENAME) and not file_exists(IMPROVED_LABELS_FILENAME):
  improved_features, improved_labels = prepare_dataset(masc_dataset)
  with open(IMPROVED_FEATURES_FILENAME, 'w') as outfile:
      json.dump(improved_features, outfile)
      
  with open(IMPROVED_LABELS_FILENAME, 'w') as outfile:
      json.dump(improved_labels, outfile)

  print ('Dumped improved_features and improved_labels')
  print ('Total features calculated:', len(improved_features))
else:
  print ('Using existing files:')

  with open(IMPROVED_FEATURES_FILENAME) as json_file:
      for sentence in json.load(json_file):
          for token in sentence:
              improved_features.append(token)


  with open(IMPROVED_LABELS_FILENAME) as json_file:
      for sentence in json.load(json_file):
          for label in sentence:
              improved_labels.append(label)

  print ('Total features loaded:', len(improved_features))

Using existing files:
Total features loaded: 2819882


# LogisticRegression on an extended set of features

In [34]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score

features_data = improved_features
labels_data = improved_labels


total_size = len(features_data)
trainset_size = round(total_size * 0.7)
testset_size = total_size - trainset_size

train_features = features_data[:trainset_size]
test_features = features_data[trainset_size:]

print ('Trainset size', len(train_features))
print('Evalset size', len(test_features))

train_labels = labels_data[:trainset_size]
test_labels = labels_data[trainset_size:]

vectorizer = DictVectorizer()
vec = vectorizer.fit(train_features)
print("Total number of features: ", len(vec.get_feature_names()))

train_features_vectorized = vec.transform(train_features)
test_features_vectorized = vec.transform(test_features)

lrc = LogisticRegression(random_state=42, solver="sag", multi_class="multinomial",
                         max_iter=100, verbose=1, n_jobs = -1)
lrc.fit(train_features_vectorized, train_labels)

scores = cross_val_score(lrc, train_features_vectorized, train_labels, cv=5)
print ('Cross validation:', scores)

predicted_labels = lrc.predict(test_features_vectorized)
print(classification_report(test_labels, predicted_labels))

Trainset size 1973917
Evalset size 845965
Total number of features:  368779


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.


max_iter reached after 254 seconds


/Users/dmytromindra/Projects/prj-nlp-2020/students/DmytroMindra/06-language-as-sequence/venv/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  4.3min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  3.3min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  3.3min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  3.3min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  3.3min finished
[Parallel(n_jobs=-1)]: Using backend LokyBacke

Cross validation: [0.98635203 0.98615445 0.98656477 0.98665089 0.98639759]
              precision    recall  f1-score   support

       False       0.99      1.00      0.99    830380
        True       0.79      0.37      0.50     15585

    accuracy                           0.99    845965
   macro avg       0.89      0.68      0.75    845965
weighted avg       0.98      0.99      0.98    845965



# LogisticRegression trained on synthetic data and validated on run_on_test.json

In [36]:
import json

def prepare_run_on_test():
    with open(RUN_ON_TEST_RAW_FILENAME) as json_file:
        data = json.load(json_file)
    labels = []
    tokens = []
    counter = 0
    for sentence in data:

        temp_tokens = []
        temp_labels = []
        for token in sentence:
            temp_tokens.append(token[0])
            temp_labels.append(token[1])

        dataset = extract_features([' '.join(temp_tokens)])

        if len(temp_tokens) != len(dataset[0]) or len(dataset[0])!=len(temp_labels):
            print('tokens:',len(temp_tokens))
            print('labels:',len(temp_labels))
            print('retokenized:',len(dataset[0]))

        if len(temp_tokens) == len(dataset[0]) and len(dataset[0])==len(temp_labels):
            tokens.append(dataset[0])
            labels.append(temp_labels)

        counter += 1
        if counter % 10 == 0:
            print('to go', len(data)-counter)

    with open(RUN_ON_TEST_TOKENS_FILENAME, 'w') as outfile:
        json.dump(tokens, outfile)
    with open(RUN_ON_TEST_LABELS_FILENAME, 'w') as outfile:
        json.dump(labels, outfile)

    return tokens, labels

BASELINE = False
run_on_features, run_on_labels = prepare_run_on_test()


tokens: 17
labels: 17
retokenized: 18
to go 190
tokens: 16
labels: 16
retokenized: 20
tokens: 36
labels: 36
retokenized: 38
tokens: 33
labels: 33
retokenized: 35
tokens: 25
labels: 25
retokenized: 27
tokens: 17
labels: 17
retokenized: 18
to go 180
tokens: 12
labels: 12
retokenized: 14
to go 170
tokens: 29
labels: 29
retokenized: 31
to go 160
to go 150
tokens: 20
labels: 20
retokenized: 21
to go 140
to go 130
to go 120
tokens: 33
labels: 33
retokenized: 37
to go 110
tokens: 17
labels: 17
retokenized: 19
to go 100
to go 90
tokens: 17
labels: 17
retokenized: 19
tokens: 56
labels: 56
retokenized: 58
to go 80
tokens: 18
labels: 18
retokenized: 20
tokens: 36
labels: 36
retokenized: 38
to go 70
tokens: 34
labels: 34
retokenized: 36
to go 60
to go 50
tokens: 20
labels: 20
retokenized: 21
tokens: 23
labels: 23
retokenized: 24
to go 40
to go 30
tokens: 20
labels: 20
retokenized: 22
to go 20
tokens: 48
labels: 48
retokenized: 49
to go 10
to go 0


In [37]:
improved_features = []
improved_labels = []

with open(IMPROVED_FEATURES_FILENAME) as json_file:
    for sentence in json.load(json_file):
        for token in sentence:
            improved_features.append(token)


with open(IMPROVED_LABELS_FILENAME) as json_file:
    for sentence in json.load(json_file):
        for label in sentence:
            improved_labels.append(label)

In [67]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score

features_data = improved_features
labels_data = improved_labels

run_on_features_data = []
run_on_labels_data = []

for sentence in run_on_features:
    for token in sentence:
        run_on_features_data.append(token)

for sentence in run_on_labels:
    for label in sentence:
        run_on_labels_data.append(label)

train_features = features_data
test_features = run_on_features_data

train_labels = labels_data
test_labels = run_on_labels_data

vectorizer = DictVectorizer()
vec = vectorizer.fit(train_features)
print("Total number of features: ", len(vec.get_feature_names()))

train_features_vectorized = vec.transform(train_features)
test_features_vectorized = vec.transform(test_features)

lrc = LogisticRegression(random_state=42, solver="sag", multi_class="multinomial",
                         max_iter=100, verbose=1, n_jobs =-1)

lrc.fit(train_features_vectorized, train_labels)

predicted_labels = lrc.predict(test_features_vectorized)
print(classification_report(test_labels, predicted_labels))

Total number of features:  441235


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


max_iter reached after 406 seconds


/Users/dmytromindra/Projects/prj-nlp-2020/students/DmytroMindra/06-language-as-sequence/venv/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  6.8min finished


              precision    recall  f1-score   support

       False       0.98      1.00      0.99      4027
        True       0.86      0.48      0.61       143

    accuracy                           0.98      4170
   macro avg       0.92      0.74      0.80      4170
weighted avg       0.98      0.98      0.98      4170



# CRF and RandomizedSearchCV

In [66]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
import sklearn_crfsuite

features_data = improved_features
labels_data = improved_labels


total_size = len(features_data)
trainset_size = round(total_size * 0.7)
testset_size = total_size - trainset_size

train_features = features_data[:trainset_size]
test_features = features_data[trainset_size:]

print ('Trainset size', len(train_features))
print('Evalset size', len(test_features))

train_labels = labels_data[:trainset_size]
test_labels = labels_data[trainset_size:]

vectorizer = DictVectorizer()
vec = vectorizer.fit(train_features)
print("Total number of features: ", len(vec.get_feature_names()))

train_features_vectorized = vec.transform(train_features)
test_features_vectorized = vec.transform(test_features)

train_labels_vectorized = []
for label in train_labels:
  if label:
    train_labels_vectorized.append(["1"])
  else:
    train_labels_vectorized.append(["0"])

crf.fit(train_features_vectorized, train_labels_vectorized)

test_labels_vectorized = []
for label in test_labels:
  if label:
    test_labels_vectorized.append(["1"])
  else:
    test_labels_vectorized.append(["0"])


crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(train_features_vectorized, train_labels_vectorized)

scores = cross_val_score(crf, train_features_vectorized, train_labels_vectorized, cv=5)
print ('Cross validation:', scores)

predicted_labels = crf.predict(test_features_vectorized)
print(classification_report(test_labels_vectorized, predicted_labels))

Trainset size 1973917
Evalset size 845965
Total number of features:  368779
Cross validation: [0.98201548 0.98192683 0.98172667 0.98220795 0.98138724]
              precision    recall  f1-score   support

           0       0.98      1.00      0.99    830380
           1       0.00      0.00      0.00     15585

    accuracy                           0.98    845965
   macro avg       0.49      0.50      0.50    845965
weighted avg       0.96      0.98      0.97    845965



In [63]:
!pip install sklearn
!pip install sklearn-crfsuite

from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
import scipy.stats
from sklearn.metrics import balanced_accuracy_score


scorer = make_scorer(balanced_accuracy_score)


crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)

params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=3,
                        n_jobs=-1,
                        n_iter=5,
                        scoring = scorer)

#train_features_vectorized = vec.transform(train_features)
#test_features_vectorized = vec.transform(test_features)

train_labels_vectorized = []
for label in train_labels:
  if label:
    train_labels_vectorized.append(["1"])
  else:
    train_labels_vectorized.append(["0"])

crf.fit(train_features_vectorized, train_labels_vectorized)

test_labels_vectorized = []
for label in test_labels:
  if label:
    test_labels_vectorized.append(["1"])
  else:
    test_labels_vectorized.append(["0"])
print (len(test_labels_vectorized))



y_pred = crf.predict(test_features_vectorized)

#print('best params:', rs.best_params_)
#print('best CV score:', rs.best_score_)
#print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

print(classification_report(test_labels_vectorized, y_pred))


4170
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      4027
           1       0.00      0.00      0.00       143

    accuracy                           0.97      4170
   macro avg       0.48      0.50      0.49      4170
weighted avg       0.93      0.97      0.95      4170

